In [14]:
from random import seed, randint
import ast; import sys; from time import process_time
import numpy as np; import verbose_module
from pickle import load, dump
from copy import deepcopy

sys.path.append('../../.')
import pIRPgym

########################     Instance generator and Environment     #########################
### pIRP model's parameters
# Stochasticity
stochastic_params = ['d','q']; look_ahead = ['d','q']

# Historical data
historical_data = ['*']

# Other parameters
backorders = 'backorders'

# Random Instance
q_params = {'dist': 'c_uniform', 'r_f_params': [80,110]}         # Supply
p_params = {'dist': 'd_uniform', 'r_f_params': [40,61]}        # Purchase costs at suppliers'

d_params = {'dist': 'log-normal-trunc', 'r_f_params': [3,1], 'break':104.5}          # Demand

h_params = {'dist': 'd_uniform', 'r_f_params': [20,40]}        # Holding costs

### Environment 
# Creating environment object
inventory = True; routing = True; perishability = "ages"
env = pIRPgym.steroid_IRP(routing, inventory, perishability)

# Environment check

In [2]:
T = 15; verbose = False; agg = False; hold_cost = True; save = True
rr = 0.1; theta = 0.90; gamma = 0.95

In [3]:
file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "rb")
feas_seeds = load(file); file.close()

In [4]:
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]
objectives = ["costs","climate","water","land","fossil"]

seed_ix = 0
for seed in feas_seeds:
    try:
        for w in weights:
            rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()
            env_config = {'M':10,'K':3,'T':T,'F':8,'Q':375,'S':5,'LA_horizon':7,'d_max':2000,'hist_window':60, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr, 'gamma':gamma}

            stoch_rd_seed = seed; det_rd_seed = 10 # Random seeds
            obj_weights = dict(zip(objectives,w))

            # Creating instance generator object
            inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
            inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

            # Resetting the environment
            states[1] = env.reset(inst_gen, return_state=True)

            done = False; day = 1
            while not done:

                ''' Objectives Pay-off Matrix and Normalization '''
                pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)
                norm_matrix[day] = deepcopy(env.norm_matrix)

                ''' Purchase '''
                [purchase,demand_compliance], la_decisions[day] = pIRPgym.Inventory.Stochastic_Rolling_Horizon(states[day], env, inst_gen, objs=obj_weights)

                ''' Routing '''
                if sum(purchase.values()) > 0:
                    GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.GeneticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)
                else:
                    GA_routes = []

                ''' Complete Action '''        
                action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

                if verbose:
                    if day == 1: verbose_module.headers.print_simple_header(w)
                    print(f"\n------- Day {day}")
                    verbose_module.objectives_performance.show_normalization(inst_gen,env)
                    verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_decisions[day])
                
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day])  = env.step(action,inst_gen,aggregated = agg)
                
                #print(f"Done day {day}")
                day += 1

            if save:
                other_path = True
                verbose_module.export_results.export_instance_parameters(w, seed_ix, inst_gen, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_actions(w, seed_ix, actions, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_lookahead_decisions(w, seed_ix, la_decisions, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_inventory(w, seed_ix, states, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_rewards(w, seed_ix, rewards, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_backorders(w, seed_ix, backo, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_perished(w, seed_ix, perished, other_path, theta=theta, gamma=gamma, min_q=rr)
                verbose_module.export_results.export_norm_matrix(w, seed_ix, norm_matrix, other_path, theta=theta, gamma=gamma, min_q=rr)
            
            print(f"\t\tDone weights {w}")
        
        print(f"DONE SEED {seed} SUCCESSFULLY")
        seed_ix += 1
    except:
        print(f"----- Failed seed {seed}")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
		Done weights [1, 0, 0, 0, 0]
		Done weights [0, 1, 0, 0, 0]
		Done weights [0, 0, 1, 0, 0]
		Done weights [0, 0, 0, 1, 0]
		Done weights [0, 0, 0, 0, 1]
DONE SEED 13854191 SUCCESSFULLY
		Done weights [1, 0, 0, 0, 0]
		Done weights [0, 1, 0, 0, 0]
		Done weights [0, 0, 1, 0, 0]
		Done weights [0, 0, 0, 1, 0]
		Done weights [0, 0, 0, 0, 1]
DONE SEED 41493042 SUCCESSFULLY
		Done weights [1, 0, 0, 0, 0]
		Done weights [0, 1, 0, 0, 0]
		Done weights [0, 0, 1, 0, 0]
		Done weights [0, 0, 0, 1, 0]
		Done weights [0, 0, 0, 0, 1]
DONE SEED 39498270 SUCCESSFULLY
		Done weights [1, 0, 0, 0, 0]
		Done weights [0, 1, 0, 0, 0]
		Done weights [0, 0, 1, 0, 0]
		Done weights [0, 0, 0, 1, 0]
		Done weights [0, 0, 0, 0, 1]
DONE SEED 1240534 SUCCESSFULLY
		Done weights [1, 0, 0, 0, 0]
		Done weights [0, 1, 0, 0, 0]
		Done weights [0, 0, 1, 0, 0]
		Done weights [0, 0, 0, 1, 0]
		Done weights [0, 0, 0, 0, 1]
DONE SE

In [5]:
file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "wb")
dump(feas_seeds, file); file.close()

# Scenarios parameter check

In [49]:
env_config = {'M':10,'K':2,'T':7,'F':8,'Q':375,'S':3,'LA_horizon':4,'d_max':2000,'hist_window':10}
st_seeds = [10,20,30,40,50]; dt_seed = 108

q_params = {'dist': 'c_uniform', 'r_f_params': [40,60]}

generators = list()
for s in st_seeds:
    generators.append(pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True))
    generators[-1].generate_random_instance(dt_seed, s, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

S = list(range(1,len(st_seeds)+1))

### Deterministic Parameters

In [50]:
verbose_module.headers.print_header("Availabilities (M_kt)")
print("\t",*[" "*3+f"s{s}"+" "*4 for s in S],sep="\t")
for k in generators[0].Products:
    print(f"\tk{k}:",*["-".join([str(i) for i in generators[s-1].M_kt[k,0]])+" "*(9-len(generators[s-1].M_kt[k,0])) for s in S],sep="\t")

verbose_module.headers.print_header("Historical Data (hist_d, hist_q, hist_p)")
print("\tDemand")
for s in S:
    print(f"\t\ts{s}:",{f"k{k}":generators[s-1].hist_d[0][k] for k in generators[0].Products},sep="\t")
print("\n\tAvailable Quantities")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":generators[s-1].hist_q[0][i,k] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")
print("\n\tPurchase Prices")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":generators[s-1].hist_p[0][i,k] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")

verbose_module.headers.print_header("Costs (c, b, h, p)")
print("\tTransport Cost")
print("\t\t",*[" "*int(np.floor((len(generators[0].V)*4-1-2)/2))+f"i{i}"+" "*int(np.floor((len(generators[0].V)*4-1-2)/2))+"   " for i in generators[-1].V],sep="\t")
for s in S:
    print(f"\t\ts{s}: ",*["-".join([" "*(3-len(str(int(generators[s-1].c[i,j]))))+str(int(generators[s-1].c[i,j])) if j != i else " 0 " for j in generators[0].V])+"\t" for i in generators[0].V], sep = "\t")

print("\n\tBackorders Cost")
print("\t\t",*[" "+f"k{k}"+" "*2 for k in generators[0].Products],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[round(generators[s-1].back_o_cost[k],2) for k in generators[0].Products],sep="\t")

print("\n\tHolding Cost")
print("\t\t",*[" "*6+f"k{k}"+" "*6  for k in generators[0].Products],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*["-".join(str(generators[s-1].W_h[t][k]) for t in generators[s-1].Horizon) for k in generators[0].Products],sep="\t")

print("\n\tPurchasing Cost")
for s in S:
    print(f"\t\ts{s}:",{f"(i{i},k{k})":[generators[s-1].W_p[t][i,k] for t in generators[0].Horizon] for k in generators[0].Products for i in generators[0].Suppliers},sep="\t")


------------------------------------------------------
                Availabilities (M_kt)                 
------------------------------------------------------

		   s1    	   s2    	   s3    	   s4    	   s5    
	k1:	1-2-3-4-7-8-10  	1-2-3-4-7-8-10  	1-2-3-4-7-8-10  	1-2-3-4-7-8-10  	1-2-3-4-7-8-10  
	k2:	1-2-5-6-7-9   	1-2-5-6-7-9   	1-2-5-6-7-9   	1-2-5-6-7-9   	1-2-5-6-7-9   

------------------------------------------------------
       Historical Data (hist_d, hist_q, hist_p)       
------------------------------------------------------

	Demand
		s1:	{'k1': [27.9, 9.06, 81.71, 4.27, 64.5, 63.29, 22.88, 30.92, 8.43, 20.64], 'k2': [4.4, 8.89, 16.06, 1.45, 21.05, 13.78, 17.07, 9.54, 58.52, 9.63]}
		s2:	{'k1': [27.9, 9.06, 81.71, 4.27, 64.5, 63.29, 22.88, 30.92, 8.43, 20.64], 'k2': [4.4, 8.89, 16.06, 1.45, 21.05, 13.78, 17.07, 9.54, 58.52, 9.63]}
		s3:	{'k1': [27.9, 9.06, 81.71, 4.27, 64.5, 63.29, 22.88, 30.92, 8.43, 20.64], 'k2': [4.4, 8.89, 16.06, 1.45, 21.05, 13.78, 17.07, 

### Stochastic Parameters

In [51]:
verbose_module.headers.print_header("Realized Demand (W_d)")
print("\t\t",*[" "*16+f"k{k}"+" "*17 for k in generators[0].Suppliers],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[[generators[s-1].W_d[t][k] for t in generators[0].Horizon] for k in generators[0].Products],sep="\t")

verbose_module.headers.print_header("Demand Sample Paths (s_paths_d)")
for t in generators[0].Horizon:
    print(f"\tDay {t}")
    for s in S:
        print(f"\t\ts{s}:",*[{f"la_t{la}":[generators[s-1].s_paths_d[t][la,ss][k] for ss in generators[0].Samples] for la in range(generators[0].sp_window_sizes[t])} for k in generators[0].Products],sep="\t")

verbose_module.headers.print_header("Realized available product (W_q)")
print("\t\t",*[" "*16+f"k{k}"+" "*17 for k in generators[0].Suppliers],sep="\t")
for s in S:
    print(f"\t\ts{s}:",*[[generators[s-1].W_q[t][i,k] for t in generators[0].Horizon] for k in generators[0].Products for i in generators[0].Suppliers],sep="\t")

verbose_module.headers.print_header("Available Quantity Sample Paths (s_paths_q)")
for t in generators[0].Horizon:
    print(f"\tDay {t}")
    for s in S:
        print(f"\t\ts{s}:",*[{f"la_t{la}":[generators[s-1].s_paths_q[t][la,ss][i,k] for ss in generators[0].Samples] for la in range(generators[0].sp_window_sizes[t])} for k in generators[0].Products for i in generators[0].Suppliers],sep="\t")



------------------------------------------------------
                Realized Demand (W_d)                 
------------------------------------------------------

			                k1                 	                k2                 	                k3                 	                k4                 	                k5                 	                k6                 	                k7                 	                k8                 	                k9                 	                k10                 
		s1:	[76.06, 4.28, 37.39, 26.19, 20.17, 30.97, 7.65]	[41.07, 19.92, 9.78, 22.39, 16.87, 66.89, 56.16]
		s2:	[48.61, 28.72, 6.79, 51.39, 33.22, 27.76, 9.1]	[24.43, 1.93, 35.15, 7.55, 30.16, 12.26, 8.65]
		s3:	[5.67, 7.61, 18.16, 3.58, 22.97, 97.34, 9.36]	[92.56, 32.15, 27.22, 98.01, 6.64, 22.37, 9.25]
		s4:	[10.94, 10.13, 3.18, 32.75, 57.72, 26.0, 48.68]	[17.71, 50.84, 12.59, 40.86, 21.2, 36.17, 7.26]
		s5:	[4.22, 10.79, 82.43, 9.2, 5.57, 22.79, 40.32]	[19.47, 4.64

# Feasible Random Seeds Search

In [5]:
verbose = False; agg = False; save = True; det_seed = 10; hold_cost=True
rr = 0.1; T = 15; theta = 0.95

In [4]:
weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]] + [[0.5,0.1,0.1,0.2,0.1]] + [[round(i*0.1,4)]+[round((1-i*0.1)/4,4) for j in range(4)] for i in range(11)]
weights = [[1,0,0,0,0],[0,1,0,0,0]]
objectives = ["costs","climate","water","land","fossil"]

rewards = dict(); actions = dict(); la_decisions = dict(); states = dict(); backo = dict(); perished = dict(); norm_matrix = dict()
env_config = {'M':10,'K':3,'T':T,'F':8,'Q':375,'S':5,'LA_horizon':7,'d_max':2000,'hist_window':60, 'theta':theta, 'hold_cost':hold_cost, 'rr':rr}

seeds = [randint(20,98765432) for i in range(2000)]

attempt = 0; feas_seeds = [13854191,41493042,39498270,1240534,40863574,22523825,21640719,39097599,21940464,36202183,12679304,29057901,
                           50500188,50124153,47042002,27623818,57665065,16394438,66275250,49680360,42595683,33529601,39543275,71017739]

while len(feas_seeds) < 50:
    try:

        stoch_rd_seed = seeds[attempt]; det_rd_seed = 10 # Random seeds

        for w in weights:
            
            
            obj_weights = dict(zip(objectives,w))

            # Creating instance generator object
            inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
            inst_gen.generate_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

            # Resetting the environment
            states[1] = env.reset(inst_gen, return_state=True)

            done = False; day = 1
            while not done:

                ''' Objectives Pay-off Matrix and Normalization '''
                pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)
                norm_matrix[day] = deepcopy(env.norm_matrix)

                ''' Purchase '''
                [purchase,demand_compliance], la_decisions[day] = pIRPgym.Inventory.Stochastic_Rolling_Horizon(states[day], env, inst_gen, objs=obj_weights)

                ''' Routing '''
                GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

                ''' Complete Action '''        
                action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

                if verbose:
                    if day == 1: verbose_module.headers.print_simple_header(w)
                    print(f"\n------- Day {day}")
                    verbose_module.objectives_performance.show_normalization(inst_gen,env)
                    verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_decisions[day])
                
                states[day+1], rewards[day], done, actions[day], (backo[day], perished[day])  = env.step(action,inst_gen,aggregated = agg)
                
                #print(f"Done day {day}")
                day += 1
            
            if save:
                other_path = True
                verbose_module.export_results.export_instance_parameters(w, 1, inst_gen, other_path)
                verbose_module.export_results.export_actions(w, 1, actions, other_path)
                verbose_module.export_results.export_lookahead_decisions(w, 1, la_decisions, other_path)
                verbose_module.export_results.export_inventory(w, 1, states, other_path)
                verbose_module.export_results.export_rewards(w, 1, rewards, other_path)
                verbose_module.export_results.export_backorders(w, 1, backo, other_path)
                verbose_module.export_results.export_perished(w, 1, perished, other_path)
                verbose_module.export_results.export_norm_matrix(w, 1, norm_matrix, other_path)
            
        feas_seeds.append(stoch_rd_seed)
        print(f"FOUND FEASIBLE SEED {len(feas_seeds)}: {stoch_rd_seed}")
        attempt += 1
                    
    except:
        
        print(f"Failed attempt {attempt} with seed {seeds[attempt]}")
        attempt += 1
        continue

file = open("C:/Users/a.rojasa55/OneDrive - Universidad de los andes/1. MIIND/Tesis/Experimentos/seeds", "wb")
dump(feas_seeds, file); file.close()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07
Failed attempt 0 with seed 44856528
FOUND FEASIBLE SEED 13: 50500188
Failed attempt 2 with seed 74348915
Failed attempt 3 with seed 73926940
FOUND FEASIBLE SEED 14: 50124153
Failed attempt 5 with seed 67077
Failed attempt 6 with seed 80781827
FOUND FEASIBLE SEED 15: 47042002
Failed attempt 8 with seed 16024188
Failed attempt 9 with seed 5106076
FOUND FEASIBLE SEED 16: 27623818
FOUND FEASIBLE SEED 17: 57665065
FOUND FEASIBLE SEED 18: 16394438
Failed attempt 13 with seed 33728435
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
FOUND FEASIBLE SEED 19: 66275250
Failed attempt 15 with seed 65517342
Failed attempt 16 with seed 18072122
Failed attempt 17 with seed 85371742
Failed attempt 18 with seed 2976801
Failed attempt 19 with seed 53122323
FOUND FEASIBLE SEED 20: 49680360
FOUND FEASIBLE SEED 

# Experiments

In [ ]:
weights = [[0.5,0.125,0.125,0.125,0.125]]; objectives = ["costs","climate","water","land","fossil"]

file = open("C:/Users/ari_r/OneDrive - Universidad de los Andes/1. MIIND/Tesis/Experimentos/seeds","rb")
seeds = load(file); file.close()

verbose = False; agg = False
for w in weights:

    for it in range(len(seeds)):

        rewards = dict()
        env_config = {'M':5,'K':7,'T':5,'F':8,'Q':1500,'S':5,'LA_horizon':4,'d_max':2000,'hist_window':60,'back_o_cost':2000}

        stoch_rd_seed, det_rd_seed = seeds[it]
        obj_weights = dict(zip(objectives,w))

        # Creating instance generator object
        inst_gen = pIRPgym.instance_generator(look_ahead, stochastic_params, historical_data, backorders, env_config = env_config, sustainability=True)
        inst_gen.generate_basic_random_instance(det_rd_seed, stoch_rd_seed, q_params=q_params, p_params=p_params, d_params=d_params, h_params=h_params)

        # Resetting the environment
        state = env.reset(inst_gen, return_state=True)

        done = False; day = 1
        while not done:

            ''' Objectives Pay-off Matrix and Normalization '''
            pIRPgym.Compromise_Programming.normalize_objectives(inst_gen, env)

            ''' Purchase '''
            [purchase,demand_compliance], la_dec = pIRPgym.Inventory.Stochastic_Rolling_Horizon(state, env, inst_gen, objs=obj_weights)

            
            ''' Routing '''
            GA_routes,GA_obj,GA_info,GA_time,_ = pIRPgym.Routing.HybridGenticAlgorithm(purchase, inst_gen, env.t, return_top=False, rd_seed=0, time_limit=30)

            ''' Complete Action '''        
            action = {'routing':GA_routes,'purchase':purchase,'demand_compliance':demand_compliance}

            if verbose:
                if day == 1: verbose_module.headers.print_simple_header(w)
                print(f"\n------- Day {day}")
                verbose_module.objectives_performance.show_normalization(inst_gen,env)
                verbose_module.objectives_performance.show_balanced_solution(inst_gen,env,la_dec)
            
            state, rewards[day], done, real_action, _,  = env.step(action,inst_gen,aggregated = agg)
            
            day += 1

        verbose_module.export_results.export_rewards(w,it+1,rewards)
        print(f"Done {w} - {it}")